In [4]:
import sys
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def load_data(boston_filepath, seattle_filepath):
    # load Boston dataset
    boston = pd.read_csv(boston_filepath)
    boston['Dataset']='Boston'
    # load Seattle dataset
    seattle = pd.read_csv(seattle_filepath)
    seattle['Dataset']='Seattle'
    # merge datasets
    for col in boston:
        if col not in seattle.columns:
            #print(col)
            boston.drop(columns=col, inplace=True)
    for col in seattle:
        if col not in boston.columns:
            #print(col)
            seattle.drop(columns=col, inplace=True)
    df = pd.concat([boston, seattle], axis=0)
    return df

def null_columns(df, show=True, drop_threshold=0.98):
    null_percent=pd.DataFrame(df.isnull().sum()/df.shape[0]).reset_index()
    null_percent.columns=['Col. Name', 'Null Percent']
    null_percent.sort_values('Null Percent', ascending=False, inplace=True)
    col_to_drop=list(null_percent[null_percent['Null Percent']>drop_threshold]['Col. Name'])
    if show:
        print(null_percent)
    
    return col_to_drop
    
def clean_listings_data(df):
    
    cat_cols=df.select_dtypes(exclude=['int64', 'float64']).columns # Categorical columns
    
    # Replace 't', 'f' with 1, 0
    for col in cat_cols:
        if {'t', 'f'}.issubset(set(df[col].unique())):
            df[col].replace({'t':1, 'f':0}, inplace=True)

    # Convert rates from string to float
    rate_cols=df.columns[df.columns.str.contains('rate')]
    df[rate_cols]=df[rate_cols].apply(lambda x: x.str.replace('%', '').astype(float)/100)
    df[rate_cols].head()
    
    # Drop some columns (these columns were having only 1 unique value in each of the 2 cities datasets, and 'host_total_listings_count' is a duplicate of another column)
    cols_to_drop=['scrape_id',
                 'last_scraped',
                 'experiences_offered',
                 'neighbourhood_group_cleansed',
                 'state',
                 'country_code',
                 'country',
                 'has_availability',
                 'calendar_last_scraped',
                 'requires_license',
                 'license',
                 'jurisdiction_names',
                 'host_total_listings_count',
                 'neighbourhood']
    df.drop(columns=cols_to_drop, inplace=True)
    
    # Rename 'neighbourhood_cleansed' after dropping 'neighbourhood'
    df.rename(columns={'neighbourhood_cleansed':'neighbourhood'}, inplace=True)
    
    # Drop columns with null values more than 98%
    df.drop(columns=null_columns(df, show=False, drop_threshold=0.95), inplace=True)
    
    # Convert Price columns to 'float' instead of 'Object'
    price_columns=[]
    for col in list(df.select_dtypes(exclude=['int64', 'float64']).columns):
        if df[col].str.contains('$', regex=False).any():
            if not df[col].str.contains('[A-Za-z]', regex=True).any():
                price_columns.append(col)
    df[price_columns]=df[price_columns].apply(lambda x:x.str.replace('[$, ]','', regex=True)).astype(float)
    
    # Create a new feature 'price_per_accommodate'
    df['price_per_accommodate']=df['price']/df['accommodates']
    
    # Split Amenities into separate columns
    df['amenities'] = df['amenities'].str.replace('[{}"]','', regex=True)
    amenities = df['amenities'].str.get_dummies(sep=',')
    df.drop(columns='amenities')
    df=pd.concat([df, amenities], axis=1)
        
    return df

def clean_calendars_data(df):
    
    # Perform datatype conversions
    df['date']=pd.to_datetime(df['date'])
    df['price']=df['price'].str.replace('[$, ]','', regex=True).astype(float)
    df['available'].replace({'t':1, 'f':0}, inplace=True)
    
    # Add new features
    df['month']=df.date.dt.month
    df['year']=df.date.dt.year
    df['month-year']=df.date.dt.strftime('%m-%Y')
    
    return df

def save_data(df, database_filename, table_name):
    engine = create_engine('sqlite:///{}'.format(database_filename))
    df.to_sql(table_name, engine, index=False, if_exists='replace')

In [5]:
df_listings=load_data('Boston/listings.csv', 'Seattle/listings.csv')
df_listings=clean_listings_data(df_listings)
df_listings.head()

,id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,city,zipcode,market,smart_location,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,Dataset,price_per_accommodate,24-Hour Check-in,Air Conditioning,Breakfast,Buzzer/Wireless Intercom,Cable TV,Carbon Monoxide Detector,Cat(s),Dog(s),Doorman,Dryer,Elevator in Building,Essentials,Family/Kid Friendly,Fire Extinguisher,First Aid Kit,Free Parking on Premises,Free Parking on Street,Gym,Hair Dryer,Hangers,Heating,Hot Tub,Indoor Fireplace,Internet,Iron,Kitchen,Laptop Friendly Workspace,Lock on Bedroom Door,Other pet(s),Paid Parking Off Premises,Pets Allowed,Pets live on this property,Pool,Safety Card,Shampoo,Smoke Detector,Smoking Allowed,Suitable for Events,TV,Washer,Washer / Dryer,Wheelchair Accessible,Wireless Internet,translation missing: en.hosting_amenity_49,translation missing: en.hosting_amenity_50
0,12147973,https://www.airbnb.com/rooms/12147973,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...","Roslindale is quiet, convenient and friendly. ...",NaN,"The bus stop is 2 blocks away, and frequent. B...",https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,31303940,https://www.airbnb.com/users/show/31303940,Virginia,2015-04-15,"Boston, Massachusetts, United States",We are country and city connecting in our deck...,NaN,NaN,NaN,0.0,https://a2.muscache.com/im/pictures/5936fef0-b...,https://a2.muscache.com/im/pictures/5936fef0-b...,Roslindale,1.0,"['email', 'phone', 'facebook', 'reviews']",1.0,0.0,"Birch Street, Boston, MA 02131, United States",Roslindale,Boston,02131,Boston,"Boston, MA",42.282619,-71.133068,1,House,Entire home/apt,4,1.5,2.0,3.0,Real Bed,"TV,Wireless Internet,Kitchen,Free Parking on P...",250.0,NaN,NaN,NaN,35.0,1,0.0,2,1125,2 weeks ago,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,moderate,0,0,1,NaN,Boston,62.50,0,0,0,0,0,0,0,1,0,1,0,1,1,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,1,0,0,1,0,0
1,3075044,https://www.airbnb.com/rooms/3075044,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,"The room is in Roslindale, a diverse and prima...","If you don't have a US cell phone, you can tex...",Plenty of safe street parking. Bus stops a few...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,2572247,https://www.airbnb.com/users/show/2572247,Andrea,2012-06-07,"Boston, Massachusetts, United States",I live in Boston and I like to travel and have...,within an hour,1.0,1.00,0.0,https://a2.muscache.com/im/users/2572247/profi...,h

In [6]:
save_data(df_listings, 'BostonSeattle.db', 'cleanedListings')

In [7]:
df_calendars=load_data('Boston/calendar.csv', 'Seattle/calendar.csv')
df_calendars=clean_calendars_data(df_calendars)
df_calendars.head()

,listing_id,date,available,price,Dataset,month,year,month-year
0,12147973,2017-09-05,0,NaN,Boston,9,2017,09-2017
1,12147973,2017-09-04,0,NaN,Boston,9,2017,09-2017
2,12147973,2017-09-03,0,NaN,Boston,9,2017,09-2017
3,12147973,2017-09-02,0,NaN,Boston,9,2017,09-2017
4,12147973,2017-09-01,0,NaN,Boston,9,2017,09-2017


In [8]:
save_data(df_calendars, 'BostonSeattle.db', 'cleanedCalendars')